# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def exp_return(data, P):
    p_in_one_year = data["Price in 1 Year"]
    probability = data ["Probability"]
    possible_return = (p_in_one_year/ P) - 1
    exp_return = (possible_return * probability).sum() 
    return exp_return

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [4]:
def volatility(data, P):
    p_in_one_year = data["Price in 1 Year"]
    probability = data ["Probability"]
    possible_return = (p_in_one_year/ P) - 1
    mean = possible_return.mean()
    var = (probability * (possible_return ** 2)).sum() - (mean ** 2)
    sd = np.sqrt(var)
    return sd

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [5]:
def interval(data, P):
    Z = 1.96 
    expected_return = exp_return(data, P)
    sd = volatility(data, P)
    N = len(data)
    margin_of_error = Z * (sd / np.sqrt(N))
    lower_bound = expected_return - margin_of_error
    upper_bound = expected_return + margin_of_error
    return [lower_bound, upper_bound]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [6]:
def exp_return_capm(beta, market_return, rf):
    expected_return_capm = rf + beta * (market_return - rf)
    return expected_return_capm

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [11]:
def clean_data(path):
    data = pd.read_csv(path)
    data = data[pd.to_numeric(data['RET'], errors='coerce').notna()]
    security_counts = data.groupby('PERMNO').size().reset_index(name='count')
    data = data[data['PERMNO'].isin(security_counts[security_counts['count'] >= 500]['PERMNO'])]
    data = data[['PERMNO', 'date', 'CUSIP', 'RET']]
    return data

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [16]:
def avg_return(data):
    
    output = pd.DataFrame(data = {
        "CUSIP" : [],
        "Average Return" : []
    })
         
    counter = 0
    total = 0
    CUSIP = data["CUSIP"][0]
    for index, row in data.iterrows():
        if (row["CUSIP"] != CUSIP):
            output.loc[len(output.index)] = [CUSIP, total / counter]
            counter = 0
            total = 0
            CUSIP = row["CUSIP"]
        total += float(row["RET"])
        counter += 1
    output.loc[len(output.index)] = [CUSIP, total / counter]
    return output

In [15]:
def security_volatilities(data):
    volatilities = data.groupby('CUSIP')['RET'].std().reset_index()
    volatilities.columns = ['CUSIP', 'Volatility']
    return volatilities  

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [14]:
def correlation(ret_data, vol_data):
    merged_data = ret_data.merge(vol_data, on='CUSIP', how='inner')
    corr = merged_data['Average Return'].corr(merged_data['Volatility'])
    return corr
    # correlation(rets, vols) should yield 0.44837090727059614